In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src')
print(sys.path)

['/home/avijendra/nl2sql_lib/nl2sql-generic/notebooks', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/avijendra/fserv/lib/python3.11/site-packages']


In [2]:
# Uncomment the below commands if required to re-install the libraries
# %pip install faiss-cpu --quiet
# %pip install google.cloud --quiet
# %pip install cloud-sql-python-connector --quiet
# %pip install pg8000 --quiet
# %pip install langchain-google-vertexai

In [3]:
import numpy as np
import csv
import faiss
from pandas import DataFrame
from datetime import datetime
from tqdm import tqdm

In [4]:
# PROJECT_ID = ! gcloud config get project
from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

# Query and Embeddings in PostgreSQL

## Initialise

In [17]:
from nl2sql_src.nl2sql_query_embeddings import PgSqlEmb

# Make sure you update the following variables in-line with your project
PGPROJ = "sl-test-project-353312"
PGLOCATION = 'us-central1'
PGINSTANCE = "test-nl2sql"
PGDB = "test-db"
PGTABLE = 'documents' #'myqueries'
PGUSER = "postgres"
PGPWD = "test-nl2sql"
# INDX_FILE = 'saved_index_pgdata' # optoinal, considered as default

pge = PgSqlEmb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)
# mytable=PGTABLE

PGSQL - class Initiated


## Create Table

In [6]:
# Create Table in PostgresDB if not already existing
pge.create_table()

['sl-test-project-353312:us-central1:test-nl2sql']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/home/avijendra/fserv/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/fserv/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno -3] Temporary failure in name resolution

The above exception was the direct cause of the following exception:

Traceback (most

## Insert records to PostgreSQL and update index

In [13]:
examples = []
with open('examples.csv', 'r') as f:
    samples_data = csv.reader(f)
    header = next(samples_data)
    for r in samples_data:
        examples.append(r)

for example in tqdm(examples):
    pge.insert_row(example[0], example[1])

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:18<00:00,  3.02s/it]


In [14]:
new_question = "How many employees in California are from Livingston County?"
mapped_sql = "select count(*) from `sl-test-project-363109.zoominfo.zi-retail-companies-headquartered-california` where ZI_C_COMPANY_COUNTY = 'Livingston County'"
pge.insert_row(new_question, mapped_sql)

## Retrieve data 

In [18]:
# Retrieve the data from the PostgreDB and return a dataframe
def extract_data():
    tmp = pge.extract_data()
    df = DataFrame(tmp.fetchall())
    print(df)
    return df

df = extract_data()
df
headers = ["question", "sql"]
df.to_csv('sample_sqls.csv', columns=headers, index=False)

    q_id                                           question  \
0      7          How many people are enrolled in CalWorks?   
1      8            How many of them live in Nevada County?   
2      9  How has participation in CalWorks changed sinc...   
3     10   How do CalWorks program participation trends ...   
4     11  How have these race and ethnicity trends chang...   
5     12  Which county has the greatest proportion of Ca...   
6     13      What about three or more additional programs?   
7     14  Which programs have the highest co-enrollment ...   
8     15  Which programs have the highest co-enrollment ...   
9     16  What county has the greatest enrollment in Cal...   
10    17  Which five counties have the lowest number of ...   
11    18  How many Hispanic individuals are served acros...   
12    19                  What is the breakdown by program?   
13    20                        Has this changed over time?   
14    21                       Change over time by prog

['sl-test-project-353312:us-central1:test-nl2sql']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/home/avijendra/fserv/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/fserv/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno -3] Temporary failure in name resolution

The above exception was the direct cause of the following exception:

Traceback (most

## Search the VectorDB for closest matching queries

In [10]:
# Search for the closest matching queries
new_query = 'What is the total revenue for Hospitality industry'
output_json = pge.search_matching_queries(new_query)
output_json

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata


[{'question': 'What is the total revenue for constuction industry',
  'sql': 'select sum(ZI_C_REVENUE) from `zoominfo.zi-companies-under-1000-employees-offering-tuition` where ZI_C_INDUSTRY_PRIMARY = "Construction"  ;'},
 {'question': 'What is the total revenue for constuction industry',
  'sql': 'select sum(ZI_C_REVENUE) from `zoominfo.zi-companies-under-1000-employees-offering-tuition` where ZI_C_INDUSTRY_PRIMARY = "Construction"  ;'},
 {'question': 'what is the revenue generated by sales agents for Manufacturing industry?',
  'sql': "select sum(ZI_C_REVENUE) as rev from `sl-test-project-363109.zoominfo.zi-companies-with-most-marketing-professionals` where ZI_C_INDUSTRY_PRIMARY = 'Manufacturing';"}]

## Clear VectorDB and re-create VectorDB from all data in PostgreSQL

In [11]:
pge.recreate_vectordb_index()

Number of elements :  14


## Clear all Data in DB

In [12]:
pge.empty_table()
df = extract_data()
df

Empty DataFrame
Columns: []
Index: []


""
